# Objetivo:
- Plotar ocorrências de focos de calor fornecidos pelo INPE num mapa interativo usando o [Leafmap](https://leafmap.org/)
- O `leafmap` é um pacote relativamente novo, versátil e amigável para mapeamento interativo de dados em Python. Desenvolvido por Qiusheng Wu como um derivado do pacote `geemap`, usado exclusivamente no contexto do **Google Earth Engine**, o `leafmap` oferece uma sintaxe simples que permite a criação de mapas interativos de forma eficiente através da integração com outros pacotes como `Folium` e `ipyleaflet`. Antes de mais nada, vamos importá-lo:

# Acesso aos dados:
- Focos de calor: https://dataserver-coids.inpe.br/queimadas/queimadas/focos/csv/
- Geojson dos municípios do Brasil: https://github.com/tbrugz/geodata-br
- Geojson do contorno de cada estado num arquivo: https://github.com/codeforgermany/click_that_hood/blob/main/public/data/brazil-states.geojson
- Geojson do contorno de cada estado em arquivos separados: https://github.com/giuliano-macedo/geodata-br-states/blob/main/geojson/br_states/

# Carregando os dados

In [ ]:
%%time
# Importa bibliotecas
from datetime import datetime, timedelta, date
import datetime
import pandas as pd

# URL dos dados do INPE
url_diario = 'https://dataserver-coids.inpe.br/queimadas/queimadas/focos/csv/diario/Brasil/'

# Loop nos dias
df = pd.DataFrame()
anos, meses, dias = [], [], []
for i in range(8):

    # gera as datas
    data = datetime.datetime.now() - timedelta(days=i)
    ano, mes, dia = date.strftime(data,'%Y'), date.strftime(data,'%m'), date.strftime(data,'%d')
    anos.append(ano)
    meses.append(mes)
    dias.append(dia)

    # leitura dos dados
    df0 = pd.read_csv(f'{url_diario}focos_diario_br_{ano}{mes}{dia}.csv')

    # junta a tabela que foi lida com a anterior
    df = pd.concat([df, df0], ignore_index=True)

# seleciona os focos de Minas Gerais
df_mg = df[df['estado']=='MINAS GERAIS']

# mostra os dados
df_mg 

# Mapa: em forma de `Pontos`

In [ ]:
# importa biblioteca
import leafmap.leafmap as leafmap

# configuração do mapa
Map = leafmap.Map(center=(-15, -60), zoom=4)

# adiciona mapa de fundo
Map.add_basemap('HYBRID')

# plota os dados
Map.add_xy_data(df_mg, x="lon", y="lat")

# adiciona contorno dos municípios do Brasil no mapa
shp_mg = 'https://raw.githubusercontent.com/tbrugz/geodata-br/master/geojson/geojs-31-mun.json'
Map.add_geojson(shp_mg, layer_name="Munícipios de MG")

# exibe o mapa
Map

In [ ]:
# salva imagem para HTML
Map.to_html("map_pontos.html")

# Mapa: em forma de `Markers`

In [ ]:
# importa biblioteca
import leafmap.leafmap as leafmap

# configuração do mapa
Map = leafmap.Map(center=(-15, -60), zoom=4)

# adiciona mapa de fundo
Map.add_basemap('HYBRID')

# plota os dados
Map.add_circle_markers_from_xy(df_mg,
                               x="lon",
                               y="lat", 
                               radius=2, 
                               color="red",
                               fill=True,
                               fill_color="red",
                               fill_opacity=1)

# adiciona contorno dos estados
shp_estados_brasil = 'https://github.com/codeforgermany/click_that_hood/blob/main/public/data/brazil-states.geojson'
Map.add_geojson(shp_estados_brasil, layer_name="Estados do Brasil")

# exibe o mapa
Map

In [ ]:
# salva imagem para HTML
Map.to_html("map_Markers.html")

# Mapa: em forma de `Heatmap`

## Leitura do dado anual do INPE

In [180]:
%%time
# Importa bibliotecas
from datetime import datetime, timedelta, date
import datetime
import pandas as pd

# URL dos dados mensais do INPE
url_mensal = 'https://dataserver-coids.inpe.br/queimadas/queimadas/focos/csv/anual/Brasil_sat_ref/'

# nome do arquivo
filename = 'focos_br_ref_2020.zip'

# leitura dos dados
df = pd.read_csv(f'{url_mensal}{filename}', compression='zip', usecols=['lat','lon','data_pas'])

# renomeia algumas colunas
df.rename(columns={'lat': 'latitude', 'lon': 'longitude'}, inplace=True)

# cria uma coluna como valor um para cada ocorrência de foco de calor
df['count'] = 1

# mostra os dados
df

CPU times: total: 594 ms
Wall time: 1.79 s


,latitude,longitude,data_pas,count
0,-3.650,-54.732,2020-06-16 17:00:00,1
1,-4.128,-47.074,2020-06-16 17:00:00,1
2,-10.513,-53.483,2020-06-16 16:55:00,1
3,-10.504,-53.498,2020-06-16 16:55:00,1
4,-10.494,-46.488,2020-06-16 16:55:00,1
...,...,...,...,...
222792,-23.163,-49.164,2020-10-02 17:15:00,1
222793,-9.558,-66.777,2020-10-13 18:40:00,1
222794,-21.691,-44.016,2020-10-13 17:00:00,1
222795,-16.451,-46.062,2020-10-13 17:00:00,1


## Plota mapa

In [181]:
# importa biblioteca
import leafmap.leafmap as leafmap

# configuração do mapa
Map = leafmap.Map(center=(-15, -60), zoom=4)

# adiciona mapa de fundo
Map.add_basemap('HYBRID')

# plota os dados
Map.add_heatmap(df,
                x="longitude",
                y="latitude",
                value="count",
                name="Heat map",
                radius=20)

# adiciona contorno dos estados
shp_estados_brasil = 'https://github.com/codeforgermany/click_that_hood/blob/main/public/data/brazil-states.geojson'
Map.add_geojson(shp_estados_brasil, layer_name="Estados do Brasil")

# exibe o mapa
Map

Map(center=[-15, -60], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [ ]:
# salva imagem para HTML
Map.to_html("map_heatmap.html")

# Mapa: para os últimos 7 dias da semana

## Processando os dados

In [195]:
%%time
# Importa bibliotecas
from datetime import datetime, timedelta, date
import datetime
import pandas as pd

# URL dos dados do INPE
url_diario = 'https://dataserver-coids.inpe.br/queimadas/queimadas/focos/csv/diario/Brasil/'

# Loop nos dias
df = pd.DataFrame()
anos, meses, dias, ano_mes_dia = [], [], [], []
for i in range(8):

    # gera as datas
    data = datetime.datetime.now() - timedelta(days=i)
    
    ano, mes, dia = date.strftime(data,'%Y'), date.strftime(data,'%m'), date.strftime(data,'%d')
    ano_mes_dia.append(f'{ano}-{mes}-{dia}')     
    anos.append(ano)
    meses.append(mes)
    dias.append(dia)

    # leitura dos dados
    df0 = pd.read_csv(f'{url_diario}focos_diario_br_{ano}{mes}{dia}.csv', usecols=['data_hora_gmt','lat','lon','satelite','municipio','estado'])

    # junta a tabela que foi lida com a anterior
    df = pd.concat([df, df0], ignore_index=True)

# ordena as datas
ano_mes_dia=sorted(ano_mes_dia)

# seleciona os focos de Minas Gerais e do Satélite AQUA
df_mg = df[ (df['estado']=='MINAS GERAIS') & (df['satelite']=='AQUA_M-T') ]

# transforma a data em índice
df_mg['data_hora_gmt'] = pd.to_datetime(df_mg['data_hora_gmt']) # transformar a coluna 'acq_date' em formato datetime
#df_mg.set_index('data_hora_gmt', inplace=True) # definir a coluna 'acq_date' como índice do DataFrame

# ordena o dataframe
#df_mg.sort_index(axis=0, inplace=True)
df_mg.sort_values(by=['data_hora_gmt'], inplace=True)

# cria a coluna com o ano, mês e dia
df_mg['Data'] = df_mg['data_hora_gmt'].apply(lambda x: x.strftime('%Y-%m-%d'))

# elimina algumas colunas
df_mg.drop(columns=['data_hora_gmt'], inplace=True)

# renomeia os nomes das colunas
df_mg.rename(columns={'lat': 'Latitude', 'lon': 'Longitude', 'satelite': 'Satélite', 'municipio': 'Município','estado': 'Estado'}, inplace=True)

# salva arquivo CSV
df_mg.to_csv('focos_diario.csv', index=False)

# mostra os dados
df_mg

CPU times: total: 219 ms
Wall time: 2.33 s


,Latitude,Longitude,Satélite,Município,Estado,Data
61201,-19.65274,-41.24707,AQUA_M-T,AIMORÉS,MINAS GERAIS,2024-06-29
61195,-21.19496,-43.84349,AQUA_M-T,BARBACENA,MINAS GERAIS,2024-06-29
61193,-20.57470,-44.86234,AQUA_M-T,CARMO DA MATA,MINAS GERAIS,2024-06-29
61191,-20.57077,-44.85529,AQUA_M-T,CARMO DA MATA,MINAS GERAIS,2024-06-29
61185,-17.91201,-42.55101,AQUA_M-T,ARICANDUVA,MINAS GERAIS,2024-06-29
...,...,...,...,...,...,...
4412,-20.63366,-43.16493,AQUA_M-T,PORTO FIRME,MINAS GERAIS,2024-07-06
4409,-16.58091,-42.40510,AQUA_M-T,VIRGEM DA LAPA,MINAS GERAIS,2024-07-06
4404,-20.63984,-43.15705,AQUA_M-T,PORTO FIRME,MINAS GERAIS,2024-07-06
4419,-15.71770,-41.02919,AQUA_M-T,DIVISÓPOLIS,MINAS GERAIS,2024-07-06


## Plota mapa
- The list of available icon names can be found at https://fontawesome.com/v4/icons.

In [197]:
# importa biblioteca
import leafmap

# configuração do mapa
Map = leafmap.Map(center=(-18, -40), zoom=6.3)

# adiciona mapa de fundo
Map.add_basemap('HYBRID')

# plota mapa
Map.add_points_from_xy('focos_diario.csv',
                       x="Longitude",
                       y="Latitude",
                       color_column="Data",                       
                       marker_colors=['lightgray', 'gray', 'blue', 'darkblue', 'green', 'darkgreen', 'red','darkred'],
                       icon_colors=['lightgray', 'gray', 'blue', 'darkblue', 'green', 'darkgreen', 'red','darkred'],
                       spin=False,
                       add_legend=True,
                       layer_name='Focos de Calor Diário')

# adiciona contorno dos municípios do Brasil no mapa
shp_mg = 'https://github.com/giuliano-macedo/geodata-br-states/blob/main/geojson/br_states/br_mg.json'
Map.add_geojson(shp_mg, layer_name="Munícipios de MG")

# exibe o mapa
Map

Map(center=[-18, -40], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…